# 3. Data Preparation – Criação da Variável Target (Churn)

Nesta etapa final da preparação dos dados, criamos a variável resposta (target) do modelo preditivo.

A variável churn representa clientes que deixaram de comprar por um período determinado, permitindo a aplicação de modelos de classificação.


In [0]:
df_rfm = spark.table("`crisp-dm`.gold_rfm")

display(df_rfm)


## 3.5 Definição da Regra de Churn

Para este projeto, definimos churn como:

Cliente que não realizou compras nos últimos 60 dias.

A escolha do período de 60 dias é comum em análises de retenção no varejo, representando um intervalo suficiente para caracterizar inatividade.


In [0]:
from pyspark.sql.functions import when

df_churn = df_rfm.withColumn(
    "churn",
    when(df_rfm.recency > 60, 1).otherwise(0)
)

display(df_churn)


## 3.6 Análise da Distribuição da Variável Target

Antes de avançar para modelagem, é fundamental verificar a proporção entre clientes ativos e clientes churn.

Essa análise permite identificar possível desbalanceamento de classes.


In [0]:
from pyspark.sql.functions import col, round

df_churn.groupBy("churn").count().withColumn(
    "percent",
    round((col("count") / df_churn.count()) * 100, 2)
).display()

Observa-se que aproximadamente 0.88% dos clientes foram classificados como churn.

Caso a proporção seja muito desbalanceada, será necessário considerar métricas como Recall e F1-Score durante a avaliação do modelo.


In [0]:
df_churn.write.mode("overwrite").saveAsTable("`crisp-dm`.gold_churn")


## Conclusão da Etapa

A variável target churn foi criada com base na métrica de recência, permitindo transformar o problema de negócio em um problema de classificação supervisionada.

O dataset Gold final contém:

- USERID
- last_purchase
- frequency
- monetary
- recency
- churn

Com isso, encerramos a etapa de Data Preparation do CRISP-DM.
